 use neural network to predict the change in the ecmwf-eps from the previous cycle run 12 hours ago for days 9-14
 ### current model features
- the difference between the new ecmwf forecast for day 9 and the previous emcwf-eps forecast for day 9[x]
- the difference between the new emcwf-eps forecast for day 8 and the previous emcwf-eps forecast for day 8[x]
- the difference between the new gfs-ens-bc forecast for days 9-14 and the previous emcwf-eps forecast for days 9-14[x]
- the difference between the new cmc-ens forecast for days 9-14 and the new gfs-ens-bc  forecast for days 9-14[x]

### new feature ideas
- rain data
- wind data
- ???

In [44]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from datetime import datetime, time

In [45]:
path = "RawData"

In [46]:
def extract_date_time(filename):
    parts = filename.split('.')
    date = parts[1]
    time = parts[2]
    return date, time

In [62]:
def get_date(df, file):
    date_str = df[df.iloc[:, 2] == 1].iloc[0]['Date']
    time_str = str(file.split('.')[2])
    date = datetime.strptime(date_str, '%Y-%m-%d')
    time_value = time(int(time_str), 0)
    combined_datetime = datetime.combine(date.date(), time_value)
    return combined_datetime

In [47]:
ecmwf_files = glob.glob(path + '/ecmwf.*.[01][02].gw_hdd.csv')
ecmwf_sorted_files = sorted(ecmwf_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[3:]

ecmwf_eps_files = glob.glob(path + '/ecmwf-eps.*.[01][02].gw_hdd.csv')
ecmwf_eps_sorted_files = sorted(ecmwf_eps_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

In [48]:
set1 = set((extract_date_time(filename) for filename in ecmwf_sorted_files))
set2 = set((extract_date_time(filename) for filename in ecmwf_eps_sorted_files))

In [49]:
ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in set2]
ecmwf_eps_sorted_files = [filename for filename in ecmwf_eps_sorted_files if extract_date_time(filename) in set1]

### step 1 get changes in ecmwf-eps compared to 12 hours ago for days 9-14

In [64]:
change_df = pd.DataFrame(columns=['diff_9', 'diff_10', 'diff_11', 'diff_12',
                                  'diff_13', 'diff_14'])

for i in range(1, len(ecmwf_eps_sorted_files)):
    ecmwf_eps_df = pd.read_csv(ecmwf_eps_sorted_files[i])
    prev_ecmwf_eps_df = pd.read_csv(ecmwf_eps_sorted_files[i-1])
    date = get_date(ecmwf_eps_df, ecmwf_eps_sorted_files[i])

    changes = []
    for day in range(8, 14):
        changes.append(ecmwf_eps_df.iloc[day]['Value'] - prev_ecmwf_eps_df.iloc[day]['Value'])
    new_row = pd.DataFrame([changes], columns=change_df.columns, index=[date])
    change_df = pd.concat([change_df, new_row])


## step 2 recreate model

### step 2.1 get difference between the new ecmwf forecast for day 9 and the previous emcwf-eps forecast for day 9

### problems:
- not all dates are in both sets of data

In [52]:
diff = {}
#for i in range(1, len(ecmwf_sorted_files)-1):
for i in range(1, 2):
    ecmwf_df = pd.read_csv(ecmwf_sorted_files[i])
    ecmwf_eps_df = pd.read_csv(ecmwf_eps_sorted_files[i-1])

    ecmwf = ecmwf_df.iloc[8]
    ecmwf_eps = ecmwf_eps_df.iloc[9]

    date = ecmwf['Date']
    diff[date] = ecmwf['Value'] - ecmwf_eps['Value']

diff_df = pd.DataFrame.from_dict(diff, orient='index', columns=['Value'])

In [58]:
ecmwf_sorted_files[1]

'RawData/ecmwf.20180711.00.gw_hdd.csv'

In [53]:
ecmwf_df

,Date,Value,Flag (0=obs 1=fcst 2=norm)
0,2018-07-04,0.057,0
1,2018-07-05,0.007,0
2,2018-07-06,0.014,0
3,2018-07-07,0.044,0
4,2018-07-08,0.007,0
5,2018-07-09,0.056,0
6,2018-07-10,0.021,0
7,2018-07-11,0.006,1
8,2018-07-12,0.004,1
9,2018-07-13,0.008,1


In [54]:
ecmwf_eps_df

,Date,Value,Flag (0=obs 1=fcst 2=norm)
0,2018-07-03,0.159,0
1,2018-07-04,0.057,0
2,2018-07-05,0.007,0
3,2018-07-06,0.014,0
4,2018-07-07,0.044,0
5,2018-07-08,0.007,0
6,2018-07-09,0.056,0
7,2018-07-10,0.021,0
8,2018-07-11,0.006,1
9,2018-07-12,0.003,1


In [41]:
diff_df

,Value
2018-07-12,0.001
